In [1]:
%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#from glob import glob
from pathlib import Path
import os, time

import numpy as np

import xarray as xr
from xorca.lib import load_xorca_dataset # not strictly necessary
from xgcm import Grid

from itidenatl.nemodez import Vmodes
#import itidenatl.eos as eos
#import itidenatl.vars as var
#import itidenatl.gridop as gop
import itidenatl.utils as ut

ImportError: dlopen(/Users/nlahaye/Coding/virtualenvs/base3.8/lib/python3.8/site-packages/cartopy/_crs.cpython-38-darwin.so, 2): Library not loaded: /usr/local/opt/proj/lib/libproj.19.dylib
  Referenced from: /Users/nlahaye/Coding/virtualenvs/base3.8/lib/python3.8/site-packages/cartopy/_crs.cpython-38-darwin.so
  Reason: image not found

In [ ]:
from dask.distributed import Client, LocalCluster
scratch = Path(os.getenv("SCRATCHDIR"))

cluster = LocalCluster(local_directory=scratch) #n_workers=24, threads_per_worker=1, memory_limit=8e6,silence_logs=50
client = Client(cluster)
client

In [ ]:
### define paths
workdir = Path("/work/CT1/ige2071/nlahaye")
scratchdir = Path(str(scratch).replace(os.getenv("USER"),"nlahaye"))
worksha = (workdir/"../SHARED").resolve()

vmod_path = {"gen": worksha/"vmodes/vmodes_10.zarr", 
             "pres": worksha/"vmodes/vmodes_10.zarr",
             "u": worksha/"vmodes/phi_u_10.zarr",
             "v": worksha/"vmodes/phi_v.zarr"
            }
             
#ssh_path = Path("/store/albert7a/eNATL60/zarr/eNATL60-BLBT02-SSH-1h") # zarr
grid_path = scratchdir/"mesh_mask_eNATL60_3.6.nc"


In [32]:
# dates and variables: will do all available if None is specified
my_dates = None
my_vars = None # list or str with "pres", "u", "v", or None for all

In [30]:
def get_modamp_path(var=None, date=None, path=None):
    """ get paths for modal projections. this should go in utils 
    Parameters
    __________
    var: str or list of str, optional
        variable name(s). None wil do all variables available
    date: str or list of str
        dates, format "yyyymmdd". None will do all available
    path: str or pathlib object, optional
        Path to modal projections. None will use default
    filename: str, optional
        name of files (can take 2 format arguments for var and date)
    
    Returns:
        dico with 
    """
    if path is None:
        path = Path("/work/CT1/ige2071/SHARED/modal_proj")
    if filename is None:
        filename = "modamp_{}_global_{}.zarr" # .format(var, date)
    if var is None:
        var = [p for p in path.glob("*") if p.is_dir()]
    if date is None:
        dico = {v:list( (path/"modamp_{}".format(v)).glob(filename.format(v, "*")) ) for v in var}
    else:
        dico = {v:[path/"modamp_{}".format(v)/filename.format(v, da) for d in list(date)] for v in var}
    return dico

In [ ]:
### load data
data_path = get_modamp_path(var=my_vars, date=my_dates)
ds = xr.Dataset()
for k,v in data_path.items():
    ds = ds.merge(xr.open_mfdataset(v, concat_dim="t", combine="nested", engine="zarr"), join="inner")
ds
chks = {k[0]:v[0] for k,v in ds.chunks.items() if k in ["x_c", "y_c"]}

### add lon, lat coords
for c in ["llon_cc", "llat_cc"]:
    if c in ds.coords:
        ds[c] = ut.open_one_coord(grid_path, chunks=chks, varname=c)[c]
    else:
        ds = ds.assign_coords({c:ut.open_one_coord(grid_path, chunks=chks, varname=c)[c]})
ds

### add a mask, from mode file
dm = xr.open_zarr(vmod_path).chunk({k+"_c":chks[k] for k in ["x","y"]})
mask = (~xr.ufuncs.isnan(dm.norm.isel(mode=0)).reset_coords(drop=True)).persist()
ds = ds.assign_coords(mask=mask)

In [27]:
plot_kwgs = {"figsize":(12,8),
                 "central_lon":-28, "extent":[-100, 2, 6, 62]
                }                 
                 
def prep_one_plot(**kwargs):
    plot_kwgs = plot_kwgs.copy()
    plot_kwgs.update(kwargs)
    map_proj = ccrs.Robinson(central_longitude=plot_kwgs["central_lon"])

    fig = plt.figure(figsize=plot_kwgs["figsize"])
    ax = plt.axes(projection=map_proj)
    gl = ax.gridlines()
    ax.add_feature(cfeature.LAND, facecolor="lightgray")
    ax.set_extent(plot_kwgs["extent"])
    return fig, ax
                     

[PosixPath('/Users/nlahaye/Documents/Zoe_drifters'),
 PosixPath('/Users/nlahaye/Documents/Docs_guides'),
 PosixPath('/Users/nlahaye/Documents/.DS_Store'),
 PosixPath('/Users/nlahaye/Documents/Thermal_RSW'),
 PosixPath('/Users/nlahaye/Documents/Past_stuff'),
 PosixPath('/Users/nlahaye/Documents/Projects'),
 PosixPath('/Users/nlahaye/Documents/Admin'),
 PosixPath('/Users/nlahaye/Documents/.#tamere.cpp'),
 PosixPath('/Users/nlahaye/Documents/Comm'),
 PosixPath('/Users/nlahaye/Documents/tiddlers_rep'),
 PosixPath('/Users/nlahaye/Documents/Bouquins'),
 PosixPath('/Users/nlahaye/Documents/Notes_misc'),
 PosixPath('/Users/nlahaye/Documents/Zotero'),
 PosixPath('/Users/nlahaye/Documents/Reviewing'),
 PosixPath('/Users/nlahaye/Documents/InternalWaves'),
 PosixPath('/Users/nlahaye/Documents/WebEx'),
 PosixPath('/Users/nlahaye/Documents/MyBox_Seafile'),
 PosixPath('/Users/nlahaye/Documents/Lens_Instab'),
 PosixPath('/Users/nlahaye/Documents/Taff_divers'),
 PosixPath('/Users/nlahaye/Documents/Docs

In [27]:
### exemple of plot
it, imod = 12, 1
pmod_moy = ds.pres.sel(mode=imod).mean("t")
toplot = (ds.pres.isel(t=it)-pmod_moy).persist().squeeze()

[PosixPath('/Users/nlahaye/Documents/Zoe_drifters'),
 PosixPath('/Users/nlahaye/Documents/Docs_guides'),
 PosixPath('/Users/nlahaye/Documents/.DS_Store'),
 PosixPath('/Users/nlahaye/Documents/Thermal_RSW'),
 PosixPath('/Users/nlahaye/Documents/Past_stuff'),
 PosixPath('/Users/nlahaye/Documents/Projects'),
 PosixPath('/Users/nlahaye/Documents/Admin'),
 PosixPath('/Users/nlahaye/Documents/.#tamere.cpp'),
 PosixPath('/Users/nlahaye/Documents/Comm'),
 PosixPath('/Users/nlahaye/Documents/tiddlers_rep'),
 PosixPath('/Users/nlahaye/Documents/Bouquins'),
 PosixPath('/Users/nlahaye/Documents/Notes_misc'),
 PosixPath('/Users/nlahaye/Documents/Zotero'),
 PosixPath('/Users/nlahaye/Documents/Reviewing'),
 PosixPath('/Users/nlahaye/Documents/InternalWaves'),
 PosixPath('/Users/nlahaye/Documents/WebEx'),
 PosixPath('/Users/nlahaye/Documents/MyBox_Seafile'),
 PosixPath('/Users/nlahaye/Documents/Lens_Instab'),
 PosixPath('/Users/nlahaye/Documents/Taff_divers'),
 PosixPath('/Users/nlahaye/Documents/Docs

In [27]:
norm = mpl.colors.Normalize(vmin=-.1, vmax=.1)
cmap = "RdBu_r"

fig, ax = prep_one_plot()
p = toplot.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), 
                                     x="llon_cc", y="llat_cc", 
                                    norm=norm, cmap=cmap)

[PosixPath('/Users/nlahaye/Documents/Zoe_drifters'),
 PosixPath('/Users/nlahaye/Documents/Docs_guides'),
 PosixPath('/Users/nlahaye/Documents/.DS_Store'),
 PosixPath('/Users/nlahaye/Documents/Thermal_RSW'),
 PosixPath('/Users/nlahaye/Documents/Past_stuff'),
 PosixPath('/Users/nlahaye/Documents/Projects'),
 PosixPath('/Users/nlahaye/Documents/Admin'),
 PosixPath('/Users/nlahaye/Documents/.#tamere.cpp'),
 PosixPath('/Users/nlahaye/Documents/Comm'),
 PosixPath('/Users/nlahaye/Documents/tiddlers_rep'),
 PosixPath('/Users/nlahaye/Documents/Bouquins'),
 PosixPath('/Users/nlahaye/Documents/Notes_misc'),
 PosixPath('/Users/nlahaye/Documents/Zotero'),
 PosixPath('/Users/nlahaye/Documents/Reviewing'),
 PosixPath('/Users/nlahaye/Documents/InternalWaves'),
 PosixPath('/Users/nlahaye/Documents/WebEx'),
 PosixPath('/Users/nlahaye/Documents/MyBox_Seafile'),
 PosixPath('/Users/nlahaye/Documents/Lens_Instab'),
 PosixPath('/Users/nlahaye/Documents/Taff_divers'),
 PosixPath('/Users/nlahaye/Documents/Docs